In [2]:
import pandas as pd

In [3]:
# Checking the dataset
df = pd.read_csv("/content/drive/MyDrive/merged_file.csv")
df

,Unnamed: 0,English,Bodo
0,0,"The city is famous for its majestic forts, pal...",बुहुमनां दाब दाब जायगानि दावबायारिफोरनि नोजोर ...
1,1,The City Palace was built by Maharaja Jai Sing...,महाराजा जय सिंह II आ बेनि महलफोरखौ लुदोंमोन आर...
2,2,"The Hawa mahal was built by the Maharaja, Sawa...",हावा हलखौ महाराजा सावाइ प्रताप सिंहवा 1799 माइ...
3,3,The Amber Fort complex has several apartments ...,"अम्बर खरङाव महल, नमा, गोजौवाव गाखोलांग्रा सिरि..."
4,4,The Amber Palace is a classic example of Mugha...,अम्बर महला मुगल आरो हिन्दु बुमिनसारनि मोनसे गो...
...,...,...,...
30355,11625,Treatment of viral hepatitis does not depend o...,भाइरेल हेपाटाइटिसनि फाहामथाया आनजादसालि आनजादन...
30356,11626,Vaccine available against hepatitis A is not c...,महामारिजों सोमोन्दो थानाय आरो बांद्राय खरसानि ...
30357,11627,Vaccine is available against hepatitis B only.,हेपाटाइटिस-बि नि थआखायल’ गोबां भेक्सिन दं।
30358,11628,Viral hepatitis may cause serious illness in p...,भाइरेल हेपाटाइटिसआ गोरबोआव थानाय आइजोखौ गोब्रा...


In [4]:
import nltk
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [19]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')

# Loading the data
df = pd.read_csv('/content/drive/MyDrive/merged_file.csv')

# Replacing missing values with an empty string
df = df.replace(np.nan, '', regex=True)

# Tokenizing and normalizing the text
bodo_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', num_words=vocab_size)
bodo_tokenizer.fit_on_texts(bodo_data)

english_data = [' '.join(word_tokenize(str(sent).lower())) for sent in df['English']]
bodo_data = [' '.join(word_tokenize(str(sent).lower())) for sent in df['Bodo']]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [44]:
# Spliting the data into training, validation, and test sets
english_train, english_test, bodo_train, bodo_test = train_test_split(english_data, bodo_data, test_size=0.2, random_state=42)
english_train, english_val, bodo_train, bodo_val = train_test_split(english_train, bodo_train, test_size=0.1, random_state=42)


In [35]:
import tensorflow as tf

# Set parameters
vocab_size = 1000
d_model = 128
num_layers = 2
num_heads = 8
dff = 512
dropout_rate = 0.1

# Define inputs
encoder_inputs = tf.keras.layers.Input(shape=(None,))
decoder_inputs = tf.keras.layers.Input(shape=(None,))

# Define embeddings
encoder_embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(encoder_inputs)
decoder_embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(decoder_inputs)

# Encode sequence
encoder_inputs = tf.keras.layers.Input(shape=(None,), name="encoder_inputs")
encoder_embeddings = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=d_model)(encoder_inputs)
encoder_embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
encoder_embeddings = tf.keras.layers.Dropout(dropout_rate)(encoder_embeddings)

encoder_output = encoder_embeddings
for i in range(num_layers):
    attention_output = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(query=encoder_output, value=encoder_output, key=encoder_output)
    attention_output = tf.keras.layers.Dropout(dropout_rate)(attention_output)
    encoder_output = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoder_output + attention_output)

encoder_output = tf.keras.layers.GlobalAveragePooling1D()(encoder_output)

encoder = tf.keras.models.Model(encoder_inputs, encoder_output, name="encoder")

    
# Decode sequence
decoder_inputs = tf.keras.layers.Input(shape=(None,), name="decoder_inputs")
num_words = len(tokenizer.word_index) + 1
decoder_embeddings = tf.keras.layers.Embedding(input_dim=num_words, output_dim=d_model)(decoder_inputs)
decoder_embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

decoder_positional_encoding = positional_encoding(maximum_position_encoding, d_model)
decoder_embeddings = decoder_embeddings + decoder_positional_encoding[:, :tf.shape(decoder_embeddings)[1], :]

decoder_output = decoder_embeddings

for i in range(num_layers):
    # Multi-head attention block
    query = tf.ones((tf.shape(decoder_output)[0], 1), dtype=tf.dtypes.int32)
    query = tf.keras.layers.Concatenate(axis=1)([query, decoder_output[:, :-1]])
    attention_output = attention([query, encoder_output])
    attention_output = tf.keras.layers.Dropout(dropout_rate)(attention_output)
    attention_output = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention_output + decoder_output)
    
    # Feedforward block
    ff_output = tf.keras.layers.Dense(units=ff_dim, activation='relu')(attention_output)
    ff_output = tf.keras.layers.Dense(units=d_model)(ff_output)
    ff_output = tf.keras.layers.Dropout(dropout_rate)(ff_output)
    ff_output = tf.keras.layers.LayerNormalization(epsilon=1e-6)(ff_output + attention_output)
    decoder_output = ff_output

decoder_output = tf.keras.layers.Dense(units=num_words, activation='softmax')(decoder_output)

# Generate output
output = tf.keras.layers.Dense(vocab_size, activation='softmax')(decoder_output)

# Define model
model = tf.keras.models.Model(inputs=[encoder_inputs, decoder_inputs], outputs=output)


NameError: ignored